## Steps to build the spelling corrector

1. Mounting the drive and loading the dataset
2. Pre-processing the data
3. Generating the candidate words
4. Defining the language model
5. Defining the channel model
6. Auto Correction

### 1. Mounting the drive and loading the dataset

In [ ]:
# importing libraries
import re
from collections import Counter

In [ ]:
# reading the dataset
file = open('big.txt').read()

In [ ]:
# content of the file
file

### 2. Pre-processing the dataset

In [ ]:
# function to extract all the words from the text file
def words(text): 
    return re.findall(r'\w+', text.lower())

In [ ]:
# extracting words from the file
words(file)

In [ ]:
# extracting the words and counting their frequency
WORDS = Counter(words(file))

In [ ]:
# words and their respective frequencies
WORDS

In [ ]:
# vocabulary size
len(WORDS)

### 3. Generating candidate words

In [ ]:
# defining a sample word
word = 'haw'

In [ ]:
# splitting the word into sub-words
splits = []
for i in range(len(word) + 1):
    splits.append((word[:i], word[i:]))

In [ ]:
# sub-words
splits

In [ ]:
# all possible letters that will be used to generate candidate words
letters = 'abcdefghijklmnopqrstuvwxyz'

In [ ]:
# generating candidates after insertion of a letter
insertion = []
for sw1, sw2 in splits:
    for c in letters:
        insertion.append(sw1 + c + sw2)

In [ ]:
# candidates after insertion
insertion

In [ ]:
# generating candidates after deletion of a letter
deletion = []
for sw1, sw2 in splits:
    if sw2:
        deletion.append(sw1 + sw2[1:])

In [ ]:
# candidates after deletion
deletion

In [ ]:
# generating candidates after substitution of a letter
substitution = []
for sw1, sw2 in splits:
    if sw2: 
        for c in letters:
            substitution.append(sw1 + c + sw2[1:])

In [ ]:
# candidates after substitution
substitution

In [ ]:
# generating candidates after transposition of two adjecent letters
transpose = []
for sw1, sw2 in splits: 
    if len(sw2)>1:
        transpose.append(sw1 + sw2[1] + sw2[0] + sw2[2:])

# candidates after transposition
transpose

In [ ]:
# all generated candidate for a word
set(insertion + deletion + substitution + transpose)

In [ ]:
# defining a function to generate all the candidates that are 1 edit distance from the original word
def edits1(word):
    # splitting the word into sub-words
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    # generating candidates after insertion of a letter
    insertion    = [sw1 + c + sw2               for sw1, sw2 in splits for c in letters]
    # generating candidates after deletion of a letter
    deletion    = [sw1 + sw2[1:]               for sw1, sw2 in splits if sw2]
    # generating candidates after substitution of a letter
    substitution   = [sw1 + c + sw2[1:]           for sw1, sw2 in splits if sw2 for c in letters]
    # generating candidates after transposition of two adjecent letters
    transpose = [sw1 + sw2[1] + sw2[0] + sw2[2:] for sw1, sw2 in splits if len(sw2)>1]
    # returning all generated candidate for a word  
    return set(deletion + insertion + substitution + transpose)

In [ ]:
# generating candidate for a sample word
edits1('the')

In [ ]:
# number of generated candidates for a sample word
len(edits1('the'))

In [ ]:
# number of generated candidates for a sample word
len(edits1('lanuage'))

In [ ]:
# function to keep only those candidates which are present in the vocabulary
def known(words):
    return set(w for w in words if w in WORDS)

In [ ]:
# generating known candidates for a sample word
known(edits1('lanuage'))

In [ ]:
# generating known candidates for a sample word
known(edits1('the'))

### 4. Defining the language Model

In [ ]:
# total number of words in the file
sum(WORDS.values())

In [ ]:
# defining a function to return the probability of a word
def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

In [ ]:
# probability of sample word
P('the')

In [ ]:
# probability of sample word
P('how')

### 5. Defining the channel model

Return a word if:
1. It is known, else
2. It is 1 edit distance away from the original word, else
3. Original word even if it is not known

In [ ]:
# function to generate the candidates
def candidates(word): 
    return list(known([word])) or list(known(edits1(word))) or [word]

In [ ]:
# generating candidates for a sample word
candidates('lanuage')

In [ ]:
# generating candidates for a sample word
candidates('the')

In [ ]:
# generating candidates for a sample word
candidates('lave')

In [ ]:
# function to pick the best word out of the generated candidates
def correction(word): 
    return max(candidates(word), key=P)

In [ ]:
# picking the best candidate for a sample word
correction('lave')

In [ ]:
# picking the best candidate for a sample word
correction('lanuage')

### 6. Auto correction 

1. Take a sentence as input
2. Generate tokens from the sentence
3. For each token, generate candidate words and return the corrected word
4. Return the auto corrected sentence




In [ ]:
# input sentence
sentence = 'how arq you'

In [ ]:
# generate tokens from the sentence
tokens = sentence.split()
tokens

In [ ]:
# generating candidates and returning the corrected word for each token
corrected_sentence = []
for i in range(len(tokens)):
    corrected_token = correction(tokens[i])
    corrected_sentence.append(corrected_token)

In [ ]:
# corrected tokens
corrected_sentence

In [ ]:
# auto corrected sentence
" ".join(corrected_sentence)

In [ ]:
# function to return the auto corrected sentence
def sentence_corrector(sentence):
    sentence = sentence.lower()
    tokens = sentence.split()
    corrected_sentence = []
    for i in range(len(tokens)):
        corrected_token = correction(tokens[i])
        corrected_sentence.append(corrected_token)
    return " ".join(corrected_sentence)

In [ ]:
# auto correcting sample sentences
sentence_corrector('natural lanuage processing')

In [ ]:
# auto correcting sample sentences
sentence_corrector('i am diing gret')

In [ ]:
# auto correcting sample sentences
sentence_corrector('are you alrikht')

In [ ]:
# auto correcting sample sentences
sentence_corrector('are you foing to the party')

## Limitations of this autocorrect model

In [ ]:
# auto correcting sample sentences
sentence_corrector('this is a big siging for us')

In [ ]:
# auto correcting sample sentences
sentence_corrector('i love how versatile acress she is')

## Solution: N-gram (bigram / trigram) language models

In [1]:
import re
from collections import Counter, defaultdict


In [2]:
# Reading the dataset
file = open('big.txt').read()

In [3]:
# Function to extract all the words from the text file
def words(text): 
    return re.findall(r'\w+', text.lower())

In [4]:
# Extracting words from the file
WORDS = Counter(words(file))


In [5]:
# Function to generate all possible splits of a word
def splits(word):
    return [(word[:i], word[i:]) for i in range(len(word) + 1)]


In [6]:
# Function to generate all possible edits that are one edit away from the original word
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    deletes    = [a + b[1:] for a, b in splits(word) if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in splits(word) if len(b) > 1]
    replaces   = [a + c + b[1:] for a, b in splits(word) if b for c in letters]
    inserts    = [a + c + b     for a, b in splits(word) for c in letters]
    return set(deletes + transposes + replaces + inserts)

# Function to keep only those candidate words which are present in the vocabulary
def known(words):
    return set(w for w in words if w in WORDS)

# Function to generate all candidate corrections for a given word
def candidates(word):
    return known([word]) or known(edits1(word)) or [word]

# Function to return the probability of a word using the trigram language model
def P(word, prev1, prev2):
    trigram = (prev1, prev2, word)
    return trigram_counts[trigram] / sum(trigram_counts[(prev1, prev2, w)] for w in WORDS)

# Function to generate all possible trigrams for a given sentence
def generate_trigrams(sentence):
    words = sentence.split()
    if len(words) < 3:
        return []
    else:
        return [(words[i], words[i+1], words[i+2]) for i in range(len(words)-2)]

# Function to correct a word using the trigram language model
def correction(word, prev1, prev2):
    return max(candidates(word), key=lambda w: P(w, prev1, prev2))



In [9]:
# Function to auto-correct a sentence using the trigram language model
def sentence_corrector(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    corrected_sentence = []

    prev1, prev2 = None, None
    for word in words:
        corrected_word = correction(word, prev1, prev2)
        corrected_sentence.append(corrected_word)
        prev1, prev2 = prev2, corrected_word
    
    return " ".join(corrected_sentence)


In [10]:
# Example usage
corrected_sentence = sentence_corrector('i love how versatile acress she is')
print("Corrected sentence:", corrected_sentence)

NameError: name 'trigram_counts' is not defined